In [1]:
import pandas as pd
import numpy as np
import json
import csv
import datetime

# PostgreSQL Database Build

In [2]:
#Load in Data
LA_raw = pd.read_csv('/Users/cathy/Documents/Columbia Sem 2/5400_Managing Data/Final Project/Code&Data/LA_Listings.csv',delimiter=',',encoding='ISO-8859-1',
                         dtype={'Street': str, 'latitude': str, 'Amenities': str, 'Calendar last scraped': str})
LA_raw['region'] = 'LA'
NY_raw = pd.read_csv('/Users/cathy/Documents/Columbia Sem 2/5400_Managing Data/Final Project/Code&Data/NY_Listings.csv',delimiter=',',encoding='ISO-8859-1',
                         dtype={'Street': str, 'Amenities': str, 'Calendar last scraped': str})
NY_raw['region'] = 'NY'

In [3]:
full_raw = pd.concat([LA_raw,NY_raw])

In [4]:
full_listings = full_raw.loc[:, ['Listing ID','Name','Host ID','Host Name','Host Response Rate','Host Is Superhost',
                                'Host total listings count','latitude','longitude','Property type','Room type','Accommodates',
                                'Bathrooms','Bedrooms','Amenities','Price','Minimum nights','Maximum nights',
                                 'Review Scores Rating']].rename(
            columns = {
                'Listing ID':'id',
                'Name':'name',
                'Host ID':'host_id',
                'Host Name':'host_name',
                'Host Response Rate':'host_response',
                'Host Is Superhost':'superhost',
                'Host total listings count':'host_listsnum',
                'Property type':'property_type',
                'Room type':'room_type',
                'Accommodates':'accommodates',
                'Bathrooms':'bathrooms',
                'Bedrooms':'bedrooms',
                'Amenities':'amenities',
                'Price':'price',
                'Minimum nights':'min_nights',
                'Maximum nights':'max_nights',
                "Review Scores Rating" : 'rating'
            })

In [5]:
full_raw['Price']    #check a specific column data type
full_raw.dtypes      #check the data type of all columns, object means string

Listing ID                       int64
Name                            object
Host ID                          int64
Host Name                       object
Host Response Rate             float64
Host Is Superhost                 bool
Host total listings count      float64
Street                          object
City                            object
Neighbourhood cleansed          object
State                           object
Country                         object
latitude                        object
longitude                      float64
Property type                   object
Room type                       object
Accommodates                   float64
Bathrooms                      float64
Bedrooms                       float64
Amenities                       object
Price                            int64
Minimum nights                   int64
Maximum nights                 float64
Availability 365                 int64
Calendar last scraped           object
Number of reviews        

#### Adjust null values

In [6]:
full_listings = full_listings.where((pd.notnull(full_listings)), None)
full_listings = full_listings.replace({np.nan: None})

In [7]:
full_listings

,id,name,host_id,host_name,host_response,superhost,host_listsnum,latitude,longitude,property_type,room_type,accommodates,bathrooms,bedrooms,amenities,price,min_nights,max_nights,rating
0,11085215,_________,57460111,Robin,None,False,3.0,34.05689088,-117.978845,House,Private room,3.0,2.0,1.0,TV;Internet;Wireless Internet;Air conditioning...,80,1,1125.0,0
1,3230382,Large master bedroom & covered patio; __________,15878447,Betty,1.0,True,9.0,34.06032707,-118.121037,House,Private room,3.0,1.0,1.0,Internet;Wireless Internet;Air conditioning;He...,75,3,1125.0,95
2,18211034,Comfortable queen bed near LAX,125720858,Sally,1.0,False,1.0,33.92183162,-118.354565,Apartment,Shared room,1.0,1.0,1.0,Internet;Wireless Internet;Pool;Kitchen;Heatin...,45,1,1125.0,0
3,12209039,Single room for rent,38037399,_,1.0,False,2.0,34.0275231,-117.809445,House,Private room,1.0,1.0,1.0,Family/kid friendly,35,3,1125.0,0
4,10339306,Temple city __________;__;__,52290241,Cynthia,0.0,False,1.0,34.09453005,-118.072263,House,Private room,4.0,1.0,1.0,None,53,1,1125.0,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75744,42806780,Male share room long term,305126403,Bruce,None,False,3.0,40.64148,-73.96073,Train,Shared room,None,None,None,None,22,30,None,0
75745,42815083,Jfk flight attendant crashpad,63729602,Adriel,None,False,1.0,40.68143,-73.75461,Cabin,Shared room,None,None,None,None,50,1,None,0
75746,42831981,Double-Bed in Sunset Park/Industry City,264569855,Leidi,None,False,3.0,40.64721,-74.01418,Bungalow,Shared room,None,None,None,None,45,1,None,0
75747,42862086,(B6) BedStuy/Bushwick Shared Basement,538108,Kevin,None,False,6.0,40.69197,-73.93003,House,Shared room,None,None,None,None,20,3,None,0


### Create Connections to PostgreSQL

In [8]:
!pip install -U "psycopg[binary]"

In [9]:
import psycopg, os

print('Connecting to the PostgreSQL database...')
conn = psycopg.connect(
    host="localhost",
    port='5432',
    dbname="5400_airbnb",
    user="postgres",
    password="123")

# create a cursor
cur = conn.cursor() #  conn object represents the connection to the database.
#A cursor is a control structure that allows you to navigate and manipulate the result set of a database query.
#It provides methods and attributes to execute SQL statements and retrieve data from the database.

Connecting to the PostgreSQL database...


In [10]:
cur.execute("DROP TABLE hosts CASCADE")
cur.execute("DROP TABLE listings CASCADE")

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost user=postgres database=5400_airbnb) at 0x7fe7f9ac1250>

In [11]:
create_tables = """
    CREATE TABLE hosts (
      host_id CHAR(50),
      host_name VARCHAR,
      host_response VARCHAR,
      superhost VARCHAR,
      host_listsnum VARCHAR,
      PRIMARY KEY (host_id)
    );

    CREATE TABLE listings (
      id INTEGER,
      name VARCHAR,
      host_id CHAR(50) REFERENCES hosts(host_id),
      latitude VARCHAR,
      longitude VARCHAR,
      property_type VARCHAR,
      room_type VARCHAR,
      accommodates INTEGER,
      bedrooms NUMERIC,
      bathrooms NUMERIC,
      price NUMERIC,
      min_nights INTEGER,
      max_nights INTEGER,
      rating NUMERIC,
      PRIMARY KEY (id)
    );
"""

cur.execute(create_tables)
conn.commit()

Insert Hosts Info

In [12]:
hosts_df = full_listings[['host_id', 'host_name','host_response','superhost','host_listsnum']]
hosts_df = hosts_df.drop_duplicates(subset='host_id')

In [13]:
for i, row in hosts_df.iterrows():
    insert_query = '''
    INSERT INTO hosts
    VALUES (%s, %s, %s, %s, %s);
    '''
    cur.execute(insert_query, (row['host_id'], row['host_name'],row['host_response'], row['superhost'],row['host_listsnum']))
conn.commit()
print("Values inserted into the hosts table successfully!")

Values inserted into the hosts table successfully!


Insert Listings Info

In [14]:
listings_df = full_listings[['id', 'name','host_id','latitude','longitude','property_type','room_type',
                            'accommodates', 'bedrooms','bathrooms','price','min_nights','max_nights','rating']]

In [15]:
for i, row in listings_df.iterrows():
    insert_query = '''
    INSERT INTO listings
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
    '''
    cur.execute(insert_query, (row['id'], row['name'],row['host_id'], row['latitude'],row['longitude'],
                              row['property_type'], row['room_type'],row['accommodates'], row['bedrooms'],
                               row['bathrooms'],row['price'], row['min_nights'],row['max_nights'],row['rating']))
conn.commit()
print("Values inserted into the listings table successfully!")

Values inserted into the listings table successfully!


In [16]:
# close the communication with the PostgreSQL
cur.close()

# MongoDB Database Build

### Reviews of Listings Data

In [17]:
# Read CSV file using pandas
reviews = pd.read_csv('/Users/cathy/Documents/Columbia Sem 2/5400_Managing Data/Final Project/Code&Data/airbnb-reviews.csv', sep=';')

reviews.drop_duplicates(subset='id', inplace=True)

In [18]:
reviews.dtypes 

listing_id        int64
id                int64
date             object
reviewer_id       int64
reviewer_name    object
comments         object
dtype: object

In [19]:
#Modify data to add location variables (long+lat) to reviews
loc_listings = full_listings.loc[:,['id','latitude','longitude']].rename(columns={"id":"listing_id"})

reviews_complete = pd.merge(reviews, loc_listings, on="listing_id")

In [20]:
reviews_complete

,listing_id,id,date,reviewer_id,reviewer_name,comments,latitude,longitude
0,6367973,139255172,2017-03-24,31402335,"Madeleine, Keylan, Skylar, Danner,",Charisse's place is the best because she makes...,40.663517,-73.960052
1,6367973,139758644,2017-03-26,95671547,Confesor,Su casa es my casa. Thanks,40.663517,-73.960052
2,6367973,142274405,2017-04-07,31402335,"Madeleine, Keylan, Skylar, Danner,","Here is the problem, when you say good thinks ...",40.663517,-73.960052
3,6367973,47409156,2015-09-18,37991741,Maritza,Charisse was very welcoming and made us feel r...,40.663517,-73.960052
4,6367973,58303193,2016-01-02,18572807,Elena,Charisse is nice and hospitable person from th...,40.663517,-73.960052
...,...,...,...,...,...,...,...,...
1214653,12410912,70074393,2016-04-16,1588333,Cris,The host canceled this reservation 2 days befo...,40.727359,-74.001918
1214654,7004813,45395202,2015-09-02,39765053,Loretta,Bright and sunny open floor plan on main floor...,36.97273496,-121.914538
1214655,7004813,44017414,2015-08-23,15790628,Jennifer,Our stay at the beach house was amazing. We ha...,36.97273496,-121.914538
1214656,5328145,26579871,2015-02-15,26769874,Ama,"Myra and David were very welcoming, friendly a...",36.97157753,-121.901532


In [21]:
modified_data = []
for listing_id, group in reviews_complete.groupby("listing_id"):
    reviews = []
    for index, row in group.iterrows():
        review = {
            "id": row["id"],
            "date": row["date"],
            "reviewer_id": row["reviewer_id"],
            "reviewer_name": row["reviewer_name"],
            "comments": row["comments"]
        }
        reviews.append(review)

    location = {
        "latitude": group["latitude"].iloc[0],
        "longitude": group["longitude"].iloc[0]
    }

    listing_data = {
        "listing_id": listing_id,
        "location": location,
        "reviews": reviews
    }

    modified_data.append(listing_data)

In [22]:
for index, row in group.iterrows():
    print("Index:", index)
    print("Row data:")
    print(row["id"])
len(modified_data)

Index: 1194569
Row data:
148920466


48852

In [23]:
#Check to see if there is any problem in the data
for item in modified_data[:2]:
    print(item)

{'listing_id': 109, 'location': {'latitude': '33.98209481', 'longitude': -118.3849352}, 'reviews': [{'id': 74506539, 'date': '2016-05-15', 'reviewer_id': 22509885, 'reviewer_name': 'Jenn', 'comments': "Me and two friends stayed for four and a half months. It was a great place to stay! The apartment was very comfortable and I really enjoyed having the park with running path across the street. The only downside was it wasn't within walking distance to restaurants, bars, or coffee shops. But they are a short drive away. Overall, great stay!"}, {'id': 449036, 'date': '2011-08-15', 'reviewer_id': 927861, 'reviewer_name': 'Christi', 'comments': 'The host canceled my reservation the day before arrival.'}]}
{'listing_id': 344, 'location': {'latitude': '34.1656163', 'longitude': -118.3345823}, 'reviews': [{'id': 79805581, 'date': '2016-06-14', 'reviewer_id': 2089550, 'reviewer_name': 'Drew & Katie', 'comments': 'We really enjoyed our stay here in Burbank! The house was clean and great for our f

In [24]:
with open('airbnb_reviews.json', 'w') as f:
    json.dump(modified_data, f)

### Create Connections to MongoDB

In [25]:
#Set up the MongoDB Client to get read for importation
import pymongo
from pymongo import MongoClient
client = MongoClient('localhost',27017) ## or MongoClient("localhost:27017")
db = client.apan5400
collection = db.airbnb_reviews

In [26]:
#Import the dataset into MongoDB Collection
collection.insert_many(modified_data)

In [27]:
#Check to see if the insertion is correct
# retrieve all documents from the collection
documents = collection.find()

# print each document
for document in documents:
    print(document)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [28]:
total_docs = collection.count_documents({})
total_docs

97704

In [29]:
client.close()